In [12]:
# basics
from datetime import datetime
import time
import os 
import random
# import gresearch_crypto

# plotting
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

# ml shit
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.stats as stats
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer
# stat shit
import statsmodels.api as sm
from statsmodels.graphics import tsaplots
# models
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor


data_root = "../data/"
SEED = 2021
REMOVE_LB_TEST_OVERLAPPING_DATA = True

In [2]:
# setting all seeds
def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

fix_all_seeds(SEED)

In [3]:
train = pd.read_csv(data_root+"train.csv")
asset_d = pd.read_csv(data_root+"asset_details.csv")

In [4]:
if REMOVE_LB_TEST_OVERLAPPING_DATA:
    train["datetime"] = pd.to_datetime(train["timestamp"], unit="s")
    train = train[train["datetime"]<"2021-06-13 00:00:00"]


In [8]:
#basic functions

#this one is prob redundant
to_datetime = lambda timestamp: datetime.strftime(datetime.fromtimestamp(timestamp),"%Y-%m-%d")

get_month_timestamp = lambda timestamp: to_datetime(timestamp).month

get_month = lambda datetime: datetime.month

get_month_day = lambda datetime: datetime.strftime("%m-%d")

############# Two new features from the competition tutorial
def log_return(series, periods=1):
    return np.log(series).diff(periods=periods)

def roll(array, shift):
    # this supposebly improves the performance of np.roll
    if not (isinstance(array, np.ndarray)):
        array = np.asarray(array)
    idx = shift%len(array)
    return np.concatenate([array[-idx:], array[:-idx]])


############# from lightGBT tutorial
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

############# realized here needs to use some data for calculation of initial values
def realized(close, N=240):
    rt = list(np.log(C_t / C_t_1) for C_t, C_t_1 in zip(close[1:], close[:-1]))
    rt_mean = sum(rt) / len(rt)
    return np.sqrt(sum((r_i - rt_mean) ** 2 for r_i in rt) * N / (len(rt) - 1))

########## function for calling all feature creating functions
def get_features(df, Lag=True):
    df_feat = df[["Count", "Open","High", "Low", "Close", "Volume","VWAP", "Target"]].copy()
    
    df_feat["Upper_shadow"] = upper_shadow(df_feat)
    df_feat["Lower_shadow"] = lower_shadow(df_feat)

    df_feat["Volume"] = log_return(df_feat["Volume"]) # maybe Volume is fine just like that(?)
    df_feat["Count"] = log_return(df_feat["Count"])
    df_feat["VWAP"] = log_return(df_feat["VWAP"])
    df_feat = df_feat[1:] # compensate
    if Lag:
        for lag in range(1, 6):
            roll_feature = "VWAP"
            df_feat["rolled_"+roll_feature+f"_{lag}"] = \
                roll(df_feat[roll_feature].values, lag)
    
    ########### for now, simple dropna()     ########### 
    ########### later we can use reindexing  ########### 
    df_feat.dropna(inplace=True)
    # df_feat = df_feat.reindex(range(btc.index[0],btc.index[-1]+60, 60), method="pad")

    return df_feat


def Xy_model_asset(train, asset_id, Lag=True):
    df = train[train["Asset_ID"]==asset_id]

    # todo : try different features here 
    #        also, scale the features
    df_proc = get_features(df, Lag)

    df_proc["y"] = df["Target"]
    # 念の為
    df_proc.dropna(how="any", inplace=True)
    X = df_proc.drop("y", axis=1)
    y = df_proc["y"]

    # todo : try different models here
    # model = LGBMRegressor()
    # model.fit(X, y)
    return X, y#, model

def weighted_correlation(a, b, weights): # a is preds, b is gt

  w = np.ravel(weights)
  a = np.ravel(a)
  b = np.ravel(b)

  sum_w = np.sum(w)
  mean_a = np.sum(a * w) / sum_w
  mean_b = np.sum(b * w) / sum_w
  var_a = np.sum(w * np.square(a - mean_a)) / sum_w
  var_b = np.sum(w * np.square(b - mean_b)) / sum_w

  cov = np.sum((a * b * w)) / np.sum(w) - mean_a * mean_b
  corr = cov / np.sqrt(var_a * var_b)

  return corr

def scorer(model, X, y):
    # not sure how this line works, just trying
    y_pred = model.predict([X])[0]
    score = np.corrcoef(y_pred, y)
    return score


my_scorer = make_scorer(scorer)

In [6]:
train_assets = train.groupby("Asset_ID")
# asset_dict = {name:train_assets[group] for name, group in zip(asset_d.Asset_Name, asset_d.Asset_ID)}

# We first create simple LGBM Baseline

In [9]:
model = LGBMRegressor()
X, y = Xy_model_asset(train, 1)
print(type(X), type(y))
scores = cross_val_score(model, X, y, cv = 5)#, scoring=scorer)
print(scores, "\n", np.mean(scores))


<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
[0.95929106 0.96415332 0.88945699 0.95953581 0.91125602] 
 0.9367386392096602


In [16]:
X

,Count,Open,High,Low,Close,Volume,VWAP,Target,Upper_shadow,Lower_shadow,rolled_VWAP_1,rolled_VWAP_2,rolled_VWAP_3,rolled_VWAP_4,rolled_VWAP_5
10,0.025864,13835.036000,14052.30000,13680.00,13828.102000,-0.016092,0.000961,-0.015037,217.264000,148.102000,0.000032,0.001131,-0.001567,-0.000113,-0.000100
18,0.809511,13823.900000,14000.40000,13601.00,13801.314000,0.572973,-0.002481,-0.010309,176.500000,200.314000,0.000961,0.000032,0.001131,-0.001567,-0.000113
26,-0.193750,13802.512000,13999.00000,13576.28,13768.040000,-0.350538,-0.001629,-0.008999,196.488000,191.760000,-0.002481,0.000961,0.000032,0.001131,-0.001567
34,0.534003,13766.000000,13955.90000,13554.44,13724.914000,1.028847,-0.003489,-0.008079,189.900000,170.474000,-0.001629,-0.002481,0.000961,0.000032,0.001131
42,-0.292185,13717.714000,14000.70000,13520.00,13717.112000,-0.426825,-0.002087,-0.004422,282.986000,197.112000,-0.003489,-0.001629,-0.002481,0.000961,0.000032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22221627,-0.105361,35512.214286,35557.00000,35495.20,35518.808571,0.006912,-0.000100,0.002982,38.191429,17.014286,-0.000669,0.000286,0.001095,0.000421,-0.002192
22221641,0.265868,35518.764114,35560.00000,35444.30,35479.547218,0.542715,-0.000113,0.002965,41.235886,35.247218,-0.000100,-0.000669,0.000286,0.001095,0.000421
22221655,0.358498,35476.735920,35500.10369,35420.32,35451.233956,-0.051945,-0.001567,0.002798,23.367770,30.913956,-0.000113,-0.000100,-0.000669,0.000286,0.001095
22221669,-0.461749,35456.720000,35540.70000,35443.09,35523.640000,-0.406577,0.001131,0.002177,17.060000,13.630000,-0.001567,-0.000113,-0.000100,-0.000669,0.000286


In [17]:
kf = KFold(n_splits=5)
model = LGBMRegressor()
scores = []
for train_idx, test_idx in kf.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores.append(np.corrcoef(y_pred, y_test))

print(scores)



[array([[1.        , 0.97950478],
       [0.97950478, 1.        ]]), array([[1.        , 0.98203391],
       [0.98203391, 1.        ]]), array([[1.        , 0.94337342],
       [0.94337342, 1.        ]]), array([[1.        , 0.97956864],
       [0.97956864, 1.        ]]), array([[1.        , 0.95460958],
       [0.95460958, 1.        ]])]


In [21]:
list = np.zeros(19)
kf.split(list)
for i, j in kf.split(list):
    print(i, j)

[ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18] [0 1 2 3]
[ 0  1  2  3  8  9 10 11 12 13 14 15 16 17 18] [4 5 6 7]
[ 0  1  2  3  4  5  6  7 12 13 14 15 16 17 18] [ 8  9 10 11]
[ 0  1  2  3  4  5  6  7  8  9 10 11 16 17 18] [12 13 14 15]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15] [16 17 18]
